In [59]:
from openbb import obb
import altair as alt
import datetime as dt
import polars as pl
import polars.selectors as cs
import datetime as dt
import infomeasure as im
start_date = dt.date(2018, 1, 1)

In [112]:
wti = obb.commodity.price.spot(provider='fred', commodity='wti', start_date=start_date)
wti = pl.DataFrame(map(lambda d: d.model_dump(), wti.results)).pivot(on='symbol', index='date')
# cpi is released once a month, downcast wti to a monthly aggregate to allow analysis
wti = wti.group_by(pl.col.date.dt.strftime("%Y-%m"), maintain_order=True).agg(cs.numeric().mean())
wti.head()

date,price_DCOILWTICO
str,f64
"""2018-01""",63.698571
"""2018-02""",62.229474
"""2018-03""",62.724762
"""2018-04""",66.25381
"""2018-05""",69.978182


In [113]:
cpi = obb.economy.cpi(country='japan,united_states,france', provider='fred', start_date=start_date)
cpi = pl.DataFrame(map(lambda d: d.model_dump(), cpi.results)).pivot(on='country', index='date')
cpi = cpi.select('date', pl.exclude('date').name.suffix('_cpi'))

# do the same as for wti
cpi = cpi.group_by(pl.col.date.dt.strftime("%Y-%m"), maintain_order=True).agg(cs.numeric().mean())
cpi.head()

date,france_cpi,japan_cpi,united_states_cpi
str,f64,f64,f64
"""2018-01""",0.013345,0.014,0.020705
"""2018-02""",0.011837,0.015,0.022118
"""2018-03""",0.015617,0.011,0.023597
"""2018-04""",0.016393,0.006,0.024627
"""2018-05""",0.020235,0.007,0.02801


In [116]:
df = wti.join(cpi, on='date', how="right").drop_nulls(subset='^.*cpi$')
print(df.shape)
df.head()

(42, 5)


price_DCOILWTICO,date,france_cpi,japan_cpi,united_states_cpi
f64,str,f64,f64,f64
63.698571,"""2018-01""",0.013345,0.014,0.020705
62.229474,"""2018-02""",0.011837,0.015,0.022118
62.724762,"""2018-03""",0.015617,0.011,0.023597
66.25381,"""2018-04""",0.016393,0.006,0.024627
69.978182,"""2018-05""",0.020235,0.007,0.02801


In [117]:
df.describe()

statistic,price_DCOILWTICO,date,france_cpi,japan_cpi,united_states_cpi
str,f64,str,f64,f64,f64
"""count""",42.0,"""42""",42.0,42.0,42.0
"""null_count""",0.0,"""0""",0.0,0.0,0.0
"""mean""",54.890751,null,0.011341,0.00319,0.020508
"""std""",12.654947,null,0.006455,0.006794,0.010213
"""min""",16.547619,"""2018-01""",-0.000191,-0.012,0.001179
"""25%""",49.522778,null,0.006738,0.001,0.015201
"""50%""",57.358095,null,0.011837,0.004,0.019102
"""75%""",63.698571,null,0.014853,0.007,0.024627
"""max""",71.378182,"""2021-06""",0.022878,0.015,0.053915


In [123]:
df.select(cs.matches("^.*cpi$").pct_change())

france_cpi,japan_cpi,united_states_cpi
f64,f64,f64
null,null,null
-0.113,0.071429,0.068238
0.319332,-0.266667,0.066876
0.049699,-0.454545,0.043663
0.23433,0.166667,0.137354
…,…,…
0.017339,-0.285714,0.197493
0.964974,-0.2,0.562903
0.121011,1.75,0.587814


In [128]:
df.select(cs.matches("^.*cpi$").pct_change())

france_cpi,japan_cpi,united_states_cpi
f64,f64,f64
null,null,null
-0.113,0.071429,0.068238
0.319332,-0.266667,0.066876
0.049699,-0.454545,0.043663
0.23433,0.166667,0.137354
…,…,…
0.017339,-0.285714,0.197493
0.964974,-0.2,0.562903
0.121011,1.75,0.587814


In [129]:
alt.data_transformers.disable_max_rows()
#cols = cs.expand_selector(df, cs.matches("^.*cpi$") | cs.by_name("commodity_DCOILWTICO").pct_change())
df.with_columns(
    cs.matches("^.*cpi$").mul(10), cs.by_name("price_DCOILWTICO").pct_change()
).unpivot(
    index='date', on=cs.matches("^.*cpi|price_DCOILWTICO$")
).plot.line(x='date', y='value', color='variable').properties(width=1000, title="")
#df.unpivot(index='date', on=pl.col.commodity_DCOILWTICO.pct_change())#.plot.line(x='date', y='value', color='variable').properties(width=700, title="")

alt.Chart(...)

In [185]:
[
    im.transfer_entropy(df.get_column('price_DCOILWTICO').pct_change(), df.get_column('france_cpi'), approach='ordinal', embedding_dim=3, prop_time=lag, base=2)
    for lag in range(0, 20)
]

[np.float64(0.798936294771064),
 np.float64(0.8173328954200425),
 np.float64(0.8737728127773259),
 np.float64(0.9603102498040896),
 np.float64(0.8023126856364046),
 np.float64(0.7082630587433577),
 np.float64(0.8360823939879122),
 np.float64(0.7092770628756254),
 np.float64(0.6984349824001042),
 np.float64(0.7250449974837702),
 np.float64(0.7809636854052712),
 np.float64(0.8577147296786966),
 np.float64(0.7837503503233395),
 np.float64(0.9058314993958982),
 np.float64(0.7624557595448118),
 np.float64(0.7627711036023674),
 np.float64(0.708978542889427),
 np.float64(0.8321139312025828),
 np.float64(0.8172028313681031),
 np.float64(0.6775809492146678)]

In [197]:
df.get_column('price_DCOILWTICO')

price_DCOILWTICO
f64
63.698571
62.229474
62.724762
66.25381
69.978182
…
59.046316
62.333043
61.716667


In [201]:
for lag in range(0, 12):
    est = im.estimator(
        df.get_column('price_DCOILWTICO'),#.pct_change().fill_null(0.0),
        df.get_column('united_states_cpi'),#.pct_change().fill_null(0.0),
        #cond=df.select('price_DCOILWTICO', b=pl.col.price_DCOILWTICO.pct_change().fill_null(0.0)),#.pct_change().fill_null(0.0),
        measure='te',
        approach='ordinal', # 'ksg'
        embedding_dim=3,
        prop_time=lag,
        base=2,
    )
    # Calculate effective transfer entropy
    #print(est.result())

    # Perform statistical testing
    stat_test = est.statistical_test(n_tests=20, method="permutation_test")
    if stat_test.p_value < .05:
        print(f'LAG: {lag}')
        effective_te = est.effective_val()
        print(f"Effective TE: {effective_te:.6f} bits")
    #print(f"90% CI: {stat_test.confidence_interval(0.9)}")
    


LAG: 5
Effective TE: -0.120584 bits
LAG: 7
Effective TE: 0.359680 bits


## TODO : optimal alignement algorithm
- we want to find a vector of positive lags maximizing TE (with p_value < 0.05)
- we also want to control overfitting

we can leverage the MDL framework for this : 
 - what is the size of data D given the model ? This is TEx->y | M; i.e the the transfer entropy of X aligned with the vector of lags and y
 - what is the size of the model itself ?
    1. `alignment.rle_id().entropy()` (kudos to data engineering) will reward aligment will long blocks of consecutive values (intuitively it's good to have a lag that's constant for a long time) + this allows comparison with the "fix-lag model"
        - question here : perhaps we want to consider `alignment.pct_change().rle_id().entropy()` to avoid penalizing linearly decreasing/increaseing blocks that points to a single shock from x leading to values in y 
    2. `aligment.entropy()` will reward alignments with a lower cardinality
    - both 1. and 2. control for the model complexity
  
 - a good model is a model that maximizes TE on the data while minimizing entropies on the alignment vector
 - with this model we can trigger early stopping and quickly discard overly complex models

How do we generate candidates to be submitted to MDL ?
 - genetic algorithms with gaussian noise ?
 - bisection
